<a href="https://colab.research.google.com/github/YolandaMDavis/cs109a-fall2020/blob/covid-research/Boston_Covid_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
DATA_ROOT = '/content/drive/My Drive/Final-Project-for-CS109a/covid_analysis/'

In [29]:
boston_covid_neighborhood_df = pd.read_csv(DATA_ROOT + 'boston_neighborhood_covid.csv')
boston_covid_neighborhood_df["total positive"] = boston_covid_neighborhood_df["total tested"] * boston_covid_neighborhood_df["cumulative positive"]
boston_covid_neighborhood_df = boston_covid_neighborhood_df.rename(columns={"neigborhood":"neighborhood"})
boston_covid_neighborhood_df.head()


,neighborhood,total tested,cumulative positive,total positive
0,East Boston,19590,0.13600,2664.2400
1,Mattapan,8652,0.11200,969.0240
2,Hyde Park,12726,0.11100,1412.5860
3,Dorchester,48850,0.10751,5251.8635
4,Roslindale,12005,0.08200,984.4100


In [30]:
boston_districts_df = pd.read_excel(DATA_ROOT + 'Boston-Neighborhoods-to-Districts.xlsx')
boston_districts_df

,Unnamed: 0,neighborhood,district
0,0,Allston,D14
1,1,Back Bay,D4
2,2,Beacon Hill,A1
3,3,Brighton,D14
4,4,Charlestown,A15
5,5,Dorchester,C11
6,6,Downtown,A1
7,7,East Boston,A7
8,8,Fenway,D4
9,9,Harbor Islands,NaN


In [31]:
boston_covid_districts = pd.merge(boston_covid_neighborhood_df,boston_districts_df, on="neighborhood").drop(columns=["Unnamed: 0"])
boston_covid_districts

,neighborhood,total tested,cumulative positive,total positive,district
0,East Boston,19590,0.136,2664.240,A7
1,Roslindale,12005,0.082,984.410,E5
2,Roxbury,23736,0.062,1471.632,B2
3,West Roxbury,9356,0.061,570.716,E5
4,South End,20060,0.057,1143.420,D4
5,Jamaica Plain,19720,0.049,966.280,E13
6,South Boston,16450,0.047,773.150,C6
7,Allston,12753,0.042,535.626,D14
8,Brighton,18697,0.042,785.274,D14
9,Charlestown,7060,0.039,275.340,A15


In [32]:
boston_covid_districts = boston_covid_districts.groupby("district").sum()[["total positive","total tested","cumulative positive"]]

In [33]:
boston_crime_df = pd.read_csv(DATA_ROOT + 'boston_crime_2020_latest.csv')
boston_crime_df.head()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,TESTTEST2,423,NaN,ASSAULT - AGGRAVATED,External,,0,2019-10-16 00:00:00,2019,10,Wednesday,0,NaN,RIVERVIEW DR,NaN,NaN,"(0.00000000, 0.00000000)"
1,S97333701,3301,NaN,VERBAL DISPUTE,C6,915,0,2020-07-18 14:34:00,2020,7,Saturday,14,NaN,MARY BOYLE WAY,42.330813,-71.051368,"(42.33081300, -71.05136800)"
2,S47513131,2647,NaN,THREATS TO DO BODILY HARM,E18,530,0,2020-06-24 10:15:00,2020,6,Wednesday,10,NaN,READVILLE ST,42.239491,-71.135954,"(42.23949100, -71.13595400)"
3,I92102201,3301,NaN,VERBAL DISPUTE,E13,583,0,2019-12-20 03:08:00,2019,12,Friday,3,NaN,DAY ST,42.325122,-71.107779,"(42.32512200, -71.10777900)"
4,I92097173,3115,NaN,INVESTIGATE PERSON,C11,355,0,2019-10-23 00:00:00,2019,10,Wednesday,0,NaN,GIBSON ST,42.297555,-71.059709,"(42.29755500, -71.05970900)"


In [39]:
crime_violence_ind = pd.read_csv(DATA_ROOT + 'boston_crime_violence_ind.csv')
crime_violence_ind["OFFENSE_TYPE"].unique()

array(['Assault and/or Battery', 'Other', 'Animal Crime',
       'MV Accident/Property Damage', 'Theft/Robbery/B&E',
       'Child Abuse/Endangerment', 'Harassment',
       'Murder/Manslaughter/Homocide', 'Drugs/Alcohol', 'Weapon',
       'Sex Offense', 'Forgery/Fraud', 'Kidnapping', 'Missing Person',
       'Prisoner Offense', 'Prostitution', 'Val', 'Violation of Order',
       'Warrant'], dtype=object)

In [35]:
# Select 2020 crimes
boston_crime_2020_df = boston_crime_df[boston_crime_df["YEAR"] == 2020].drop(columns=["OFFENSE_CODE_GROUP"])

# Merge with Covid Data
boston_crime_2020_df = pd.merge(boston_crime_2020_df,boston_covid_districts, left_on="DISTRICT", right_on="district")

# Merge with new Violent Crime Indicator
boston_crime_2020_df = pd.merge(boston_crime_2020_df,crime_violence_ind, left_on="OFFENSE_DESCRIPTION", right_on="OFFENSE_DESCRIPTION")

# Set any shooting indicator as violent crime
filter = boston_crime_2020_df["SHOOTING"] == 1
boston_crime_2020_df.loc[filter,"VIOLENT_CRIME"] = 1


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,total positive,total tested,cumulative positive,VIOLENT_CRIME,OFFENSE_TYPE
384,202076827,3301,VERBAL DISPUTE,B2,266,1,2020-10-18 14:50:00,2020,10,Sunday,14,NaN,WINTHROP ST,42.324240,-71.077619,"(42.32424000, -71.07761900)",1471.632,23736,0.062,1,Other
5299,202016431,3115,INVESTIGATE PERSON,C6,914,1,2020-02-29 12:34:00,2020,2,Saturday,12,NaN,NaN,42.325421,-71.054182,"(42.32542100, -71.05418200)",773.150,16450,0.047,1,Other
5393,202077826,3115,INVESTIGATE PERSON,B2,,1,2020-10-21 21:12:00,2020,10,Wednesday,21,NaN,NaN,42.310037,-71.097935,"(42.31003700, -71.09793500)",1471.632,23736,0.062,1,Other
5536,202064743,3115,INVESTIGATE PERSON,B2,,1,2020-09-05 20:48:00,2020,9,Saturday,20,NaN,NaN,42.324925,-71.098396,"(42.32492500, -71.09839600)",1471.632,23736,0.062,1,Other
5588,202060175,3115,INVESTIGATE PERSON,B2,,1,2020-08-20 19:32:00,2020,8,Thursday,19,NaN,NaN,42.307609,-71.083906,"(42.30760900, -71.08390600)",1471.632,23736,0.062,1,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50051,202001114,2622,KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION,A15,900,0,2020-01-05 08:22:00,2020,1,Sunday,8,NaN,BUNKER HILL ST,42.377363,-71.057913,"(42.37736300, -71.05791300)",275.340,7060,0.039,1,Kidnapping
50052,202012821,2622,KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION,D14,788,0,2020-02-16 00:50:00,2020,2,Sunday,0,NaN,COMMONWEALTH AVE,42.341368,-71.148646,"(42.34136800, -71.14864600)",1320.900,31450,0.084,1,Kidnapping
50053,202010818,2622,KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION,A1,125,0,2020-02-09 03:00:00,2020,2,Sunday,3,NaN,WINCHESTER ST,42.349503,-71.068904,"(42.34950300, -71.06890400)",476.523,17649,0.108,1,Kidnapping
50054,202049892,2622,KIDNAPPING/CUSTODIAL KIDNAPPING/ ABDUCTION,A7,,0,2020-07-15 09:40:00,2020,7,Wednesday,9,NaN,BORDER ST,42.376202,-71.039799,"(42.37620200, -71.03979900)",2664.240,19590,0.136,1,Kidnapping
